In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV  # Import RandomizedSearchCV
from sklearn.metrics import make_scorer  # Import make_scorer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
import catboost as cb
import os

In [36]:
train = pd.read_csv("Dataset/DataEdbert/baseline_data_imputed.csv")
test = pd.read_csv("Dataset/DataEdbert/baseline_data_test.csv")

In [37]:
train["node_awal_ways_node_akhir_ways"] = train["node_awal_ways"] - train["node_akhir_ways"]
test["node_awal_ways_node_akhir_ways"] = test["node_awal_ways"] - test["node_akhir_ways"]

In [38]:
test.head()

,id,id_jalan,id_titik_mulai,id_titik_akhir,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,hour_group,id_jalan_id_titik_mulai_id_titik_akhir,is_weekend,node_awal_ways_node_akhir_ways
0,0,4004732,32046542,6454026544,23,0,West,3.0,3.0,9,7,30,trunk,1,4004732_32046542_6454026544,True,0.0
1,1,182210371,1314925464,1314925496,23,0,Side,2.0,2.0,13,11,30,trunk,1,182210371_1314925464_1314925496,True,0.0
2,2,22932408,1482086782,26481020,23,0,Lane,2.0,2.0,4,62,20,secondary,1,22932408_1482086782_26481020,True,0.0
3,3,182210371,3892883,267337489,23,0,Side,2.0,2.0,13,11,30,trunk,1,182210371_3892883_267337489,True,0.0
4,4,66924592,266041030,2592978110,23,0,Road,2.0,3.0,25,34,30,trunk,1,66924592_266041030_2592978110,True,-1.0


In [39]:
submission = pd.read_csv("Dataset/sample_submission.csv")
submission.head()

,id,rerata_kecepatan
0,0,35
1,1,35
2,2,35
3,3,35
4,4,35


In [40]:
train.shape, test.shape

((493152, 18), (127489, 17))

In [41]:
set(train.columns).difference(test.columns)

{'hari', 'rerata_kecepatan'}

In [42]:
mapping = {
    1: "Sabtu",
    2: "Minggu",
    3: "Senin",
    4: "Selasa",
    5: "Rabu",
    6: "Kamis",
    7: "Jumat",
    8: "Sabtu",
    9: "Minggu",
    10: "Senin",
    11: "Selasa",
    12: "Rabu",
    13: "Kamis",
    14: "Jumat",
    15: "Sabtu",
    16: "Minggu",
    17: "Senin",
    18: "Selasa",
    19: "Rabu",
    20: "Kamis",
    21: "Jumat",
    22: "Sabtu",
    23: "Minggu",
    24: "Senin",
    25: "Selasa",
    26: "Rabu",
    27: "Kamis",
    28: "Jumat",
    29: "Sabtu"
}

test["hari"] = test["Date"].map(mapping)

In [44]:
train.head()

,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,hour_group,id_jalan_id_titik_mulai_id_titik_akhir,is_weekend,hari,node_awal_ways_node_akhir_ways
0,691007296.0,21390008.0,1.425033e+09,29.126,1,0,Road,3.0,3.0,13.0,5.0,30.0,trunk,1.0,691007296_21390008_1425033102,True,Sabtu,0.0
1,691007296.0,21390008.0,1.425033e+09,29.126,1,1,Road,3.0,3.0,13.0,5.0,30.0,trunk,1.0,691007296_21390008_1425033102,True,Sabtu,0.0
2,691007296.0,21390008.0,1.425033e+09,34.772,1,2,Road,3.0,3.0,13.0,5.0,30.0,trunk,1.0,691007296_21390008_1425033102,True,Sabtu,0.0
3,691007296.0,21390008.0,1.425033e+09,34.161,1,3,Road,3.0,3.0,13.0,5.0,30.0,trunk,1.0,691007296_21390008_1425033102,True,Sabtu,0.0
4,691007296.0,21390008.0,1.425033e+09,39.383,1,4,Road,3.0,3.0,13.0,5.0,30.0,trunk,1.0,691007296_21390008_1425033102,True,Sabtu,0.0


In [10]:
test.head()

,id,id_jalan,id_titik_mulai,id_titik_akhir,Date,Hour,road_type,node_awal_ways,node_akhir_ways,road_relation,road_node,max_speed,type_highway,hour_group,id_jalan_id_titik_mulai_id_titik_akhir,is_weekend,node_awal_ways_node_akhir_ways,hari
0,0,4004732,32046542,6454026544,23,0,West,3.0,3.0,9,7,30,trunk,1,4004732_32046542_6454026544,True,0.0,Minggu
1,1,182210371,1314925464,1314925496,23,0,Side,2.0,2.0,13,11,30,trunk,1,182210371_1314925464_1314925496,True,0.0,Minggu
2,2,22932408,1482086782,26481020,23,0,Lane,2.0,2.0,4,62,20,secondary,1,22932408_1482086782_26481020,True,0.0,Minggu
3,3,182210371,3892883,267337489,23,0,Side,2.0,2.0,13,11,30,trunk,1,182210371_3892883_267337489,True,0.0,Minggu
4,4,66924592,266041030,2592978110,23,0,Road,2.0,3.0,25,34,30,trunk,1,66924592_266041030_2592978110,True,-1.0,Minggu


In [11]:
def remove_dot_zero(x):
    if isinstance(x, float) and x.is_integer():
        return str(int(x))
    return x

# Apply the function to each element of the DataFrame
train = train.applymap(remove_dot_zero)
test = test.applymap(remove_dot_zero)

In [12]:
cat = ["id_jalan","id_titik_mulai","id_titik_akhir","Date","Hour","road_type","max_speed","type_highway","hour_group","id_jalan_id_titik_mulai_id_titik_akhir","hari"]
for z in cat:
    train[z] = train[z].astype("category")
    test[z] = test[z].astype("category")

In [13]:
train["node_awal_ways"].isna().sum()

18480

In [14]:
import pandas as pd

# Assuming you have already loaded your train and test DataFrames

# Fill missing values with the median and convert to numeric values
train["node_awal_ways"] = pd.to_numeric(train["node_awal_ways"], errors='coerce').fillna(train["node_awal_ways"].median()).astype(int)
train["node_akhir_ways"] = pd.to_numeric(train["node_akhir_ways"], errors='coerce').fillna(train["node_akhir_ways"].median()).astype(int)

test["node_awal_ways"] = pd.to_numeric(test["node_awal_ways"], errors='coerce').fillna(test["node_awal_ways"].median()).astype(int)
test["node_akhir_ways"] = pd.to_numeric(test["node_akhir_ways"], errors='coerce').fillna(test["node_akhir_ways"].median()).astype(int)


In [15]:
print(test['id_jalan_id_titik_mulai_id_titik_akhir'].nunique())

934


In [16]:
# train = train.drop(["id_jalan","id_titik_mulai","id_titik_akhir"],axis = 1)
# test = test.drop(["id_jalan","id_titik_mulai","id_titik_akhir"],axis = 1)

In [17]:
train[["road_relation","road_node"]] = train[["road_relation","road_node"]].astype(int)
train["rerata_kecepatan"] = train["rerata_kecepatan"].astype(float)

In [18]:
cat = ["id_jalan","id_titik_mulai","id_titik_akhir","Date","Hour","road_type","max_speed","type_highway","hour_group","id_jalan_id_titik_mulai_id_titik_akhir","hari"]

In [19]:
# # Define the Symmetric Mean Absolute Percentage Error (SMAPE) custom scoring function
# def smape(y_true, y_pred):
#     """
#     Compute the Symmetric Mean Absolute Percentage Error.
#     """
#     denominator = (np.abs(y_true) + np.abs(y_pred))
#     diff = np.abs(y_true - y_pred) / denominator
#     return np.mean(2 * diff)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(train.drop("rerata_kecepatan", axis=1),
#                                                     train["rerata_kecepatan"],
#                                                     test_size=0.2,
#                                                     random_state=42)

# # Create the CatBoost model with GPU support
# catboost_model = CatBoostRegressor(task_type='GPU', cat_features=cat, random_seed=42)

# # Define the hyperparameter grid for RandomizedSearchCV
# param_dist = {
#     'iterations': [1000, 2000, 3000],
#     'depth': [6, 8, 10],
#     'learning_rate': [0.8, 0.9, 1],
# }

# # Create the RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=catboost_model,
#                                    param_distributions=param_dist,
#                                    n_iter=10,
#                                    scoring=make_scorer(smape, greater_is_better=False),
#                                    cv=3,
#                                    verbose=500)

# # Perform the random search for hyperparameter tuning
# random_search.fit(X_train, y_train, verbose=500)

# # Get the best CatBoost model with the best hyperparameters
# best_catboost_model = random_search.best_estimator_

# # Make predictions on the test set using the best model
# y_pred = best_catboost_model.predict(X_test)

# # Calculate and print the SMAPE as a performance metric
# smape_score = smape(y_test, y_pred)
# print("Best Hyperparameters:", random_search.best_params_)
# print("SMAPE Score on Test Set:", smape_score)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(train.drop("rerata_kecepatan", axis=1),
                                                    train["rerata_kecepatan"],
                                                    test_size=0.2,
                                                    random_state=42)

In [21]:
# clf = cb.CatBoostRegressor(
#     iterations=2000,
#     verbose=500,
#     depth=10, 
#     learning_rate=0.8,
#     loss_function="MAPE",
#     task_type="GPU"
# )

# clf.fit(
#     X_train, y_train,
#     cat_features=cat,
#     eval_set=(X_val, y_val),
# )

In [22]:
# bestTest = 0.1010462331
# bestTest = 0.0845867425 (2000, 10, 0.8)
# bestTest = 0.08192791147 (2000, 10, 1)
# bestTest (with ext) = 0.08416484366 (2000, 10, 0.8)
# bestTest (with ext) = 0.08178536429 (2000, 10, 1)

In [23]:
train["node_awal_ways_node_akhir_ways"] = train["node_awal_ways"] - train["node_akhir_ways"]
test["node_awal_ways_node_akhir_ways"] = test["node_awal_ways"] - test["node_akhir_ways"]

In [24]:
def extract_first_three_digits(number):
    number_str = str(number)
    first_three_digits = number_str[:3]
    return int(first_three_digits)

def extract_first_five_digits(number):
    number_str = str(number)
    first_three_digits = number_str[:4]
    return int(first_three_digits)


train['id_jalan_3'] = train['id_jalan'].apply(extract_first_three_digits)
train['id_titik_mulai_4'] = train['id_titik_mulai'].apply(extract_first_five_digits)
train['id_titik_akhir_4'] = train['id_titik_akhir'].apply(extract_first_five_digits)

test['id_jalan_3'] = test['id_jalan'].apply(extract_first_three_digits)
test['id_titik_mulai_4'] = test['id_titik_mulai'].apply(extract_first_five_digits)
test['id_titik_akhir_4'] = test['id_titik_akhir'].apply(extract_first_five_digits)

In [25]:
X_train = train.drop("rerata_kecepatan", axis=1)
y_train = train["rerata_kecepatan"]
X_test = test.drop("id", axis=1)
X_test = X_test.dropna()
ids = test["id"]

In [26]:
X_train, X_test = X_train.align(X_test)

In [27]:
X_train.head()

,Date,Hour,hari,hour_group,id_jalan,id_jalan_3,id_jalan_id_titik_mulai_id_titik_akhir,id_titik_akhir,id_titik_akhir_4,id_titik_mulai,id_titik_mulai_4,is_weekend,max_speed,node_akhir_ways,node_awal_ways,node_awal_ways_node_akhir_ways,road_node,road_relation,road_type,type_highway
0,1,0,Sabtu,1,691007296,691,691007296_21390008_1425033102,1425033102,1425,21390008,2139,True,30,3,3,0,5,13,Road,trunk
1,1,1,Sabtu,1,691007296,691,691007296_21390008_1425033102,1425033102,1425,21390008,2139,True,30,3,3,0,5,13,Road,trunk
2,1,2,Sabtu,1,691007296,691,691007296_21390008_1425033102,1425033102,1425,21390008,2139,True,30,3,3,0,5,13,Road,trunk
3,1,3,Sabtu,1,691007296,691,691007296_21390008_1425033102,1425033102,1425,21390008,2139,True,30,3,3,0,5,13,Road,trunk
4,1,4,Sabtu,1,691007296,691,691007296_21390008_1425033102,1425033102,1425,21390008,2139,True,30,3,3,0,5,13,Road,trunk


In [28]:
X_test.head()

,Date,Hour,hari,hour_group,id_jalan,id_jalan_3,id_jalan_id_titik_mulai_id_titik_akhir,id_titik_akhir,id_titik_akhir_4,id_titik_mulai,id_titik_mulai_4,is_weekend,max_speed,node_akhir_ways,node_awal_ways,node_awal_ways_node_akhir_ways,road_node,road_relation,road_type,type_highway
0,23,0,Minggu,1,4004732,400.0,4004732_32046542_6454026544,6454026544,6454.0,32046542,3204.0,True,30,3.0,3.0,0.0,7.0,9.0,West,trunk
1,23,0,Minggu,1,182210371,182.0,182210371_1314925464_1314925496,1314925496,1314.0,1314925464,1314.0,True,30,2.0,2.0,0.0,11.0,13.0,Side,trunk
2,23,0,Minggu,1,22932408,229.0,22932408_1482086782_26481020,26481020,2648.0,1482086782,1482.0,True,20,2.0,2.0,0.0,62.0,4.0,Lane,secondary
3,23,0,Minggu,1,182210371,182.0,182210371_3892883_267337489,267337489,2673.0,3892883,3892.0,True,30,2.0,2.0,0.0,11.0,13.0,Side,trunk
4,23,0,Minggu,1,66924592,669.0,66924592_266041030_2592978110,2592978110,2592.0,266041030,2660.0,True,30,3.0,2.0,-1.0,34.0,25.0,Road,trunk


In [29]:
y_train[:5]

0    29.126
1    29.126
2    34.772
3    34.161
4    39.383
Name: rerata_kecepatan, dtype: float64

In [30]:
X_test = X_test.dropna()
X_test.isna().sum()

Date                                      0
Hour                                      0
hari                                      0
hour_group                                0
id_jalan                                  0
id_jalan_3                                0
id_jalan_id_titik_mulai_id_titik_akhir    0
id_titik_akhir                            0
id_titik_akhir_4                          0
id_titik_mulai                            0
id_titik_mulai_4                          0
is_weekend                                0
max_speed                                 0
node_akhir_ways                           0
node_awal_ways                            0
node_awal_ways_node_akhir_ways            0
road_node                                 0
road_relation                             0
road_type                                 0
type_highway                              0
dtype: int64

In [31]:
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

clf = cb.CatBoostRegressor(
    iterations=3000,
    verbose=100,
    depth=10,
    learning_rate=1,
    loss_function="MAPE",
    task_type="GPU",
    random_seed=42
)

clf.fit(
    X_train, y_train,
    cat_features=cat
)

0:	learn: 0.2284040	total: 89.2ms	remaining: 4m 27s
100:	learn: 0.1784484	total: 8.47s	remaining: 4m 3s
200:	learn: 0.1453777	total: 18.1s	remaining: 4m 11s
300:	learn: 0.1246683	total: 28.5s	remaining: 4m 15s
400:	learn: 0.1135517	total: 38s	remaining: 4m 5s
500:	learn: 0.1064165	total: 47s	remaining: 3m 54s
600:	learn: 0.1021981	total: 56.8s	remaining: 3m 46s
700:	learn: 0.0995620	total: 1m 7s	remaining: 3m 40s
800:	learn: 0.0975726	total: 1m 17s	remaining: 3m 32s
900:	learn: 0.0957832	total: 1m 28s	remaining: 3m 25s
1000:	learn: 0.0940360	total: 1m 39s	remaining: 3m 18s
1100:	learn: 0.0923606	total: 1m 50s	remaining: 3m 11s
1200:	learn: 0.0909001	total: 2m 2s	remaining: 3m 3s
1300:	learn: 0.0897028	total: 2m 13s	remaining: 2m 54s
1400:	learn: 0.0886071	total: 2m 25s	remaining: 2m 46s
1500:	learn: 0.0876633	total: 2m 37s	remaining: 2m 37s
1600:	learn: 0.0867675	total: 2m 50s	remaining: 2m 28s
1700:	learn: 0.0860501	total: 3m 1s	remaining: 2m 18s
1800:	learn: 0.0853731	total: 3m 13s	r

In [32]:
y_pred = clf.predict(X_test)
y_pred

array([42.24120876, 40.96286853, 36.56331061, ..., 42.73386193,
       35.12456818, 43.4972032 ])

In [33]:
d = {
    "id":ids,
    "rerata_kecepatan":y_pred
}

df = pd.DataFrame(data=d)
df.head(20)

,id,rerata_kecepatan
0,0,42.241209
1,1,40.962869
2,2,36.563311
3,3,42.458925
4,4,28.902826
5,5,29.731497
6,6,44.195642
7,7,33.553594
8,8,30.957037
9,9,41.749280


In [34]:
df.to_csv("Submission/cb_fe_impute_overfit.csv", index=False)